<a href="https://colab.research.google.com/github/XekoFrontend/google-colab-notebook/blob/main/get_truyen3sqq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import requests
from bs4 import BeautifulSoup
import os
import time
from urllib.parse import urlparse
from zipfile import ZipFile

# ==== CONFIG ====
URL = "https://truyentranh3qq.com/vung-hoang-mac/chuong-2"  # <-- Đổi sang link chap mày muốn
SAVE_DIR = "downloaded_images"
ZIP_FILE = "chapter.cbz"  # Nếu không cần zip thì bỏ qua

# Tạo thư mục
os.makedirs(SAVE_DIR, exist_ok=True)

# Lấy HTML trang
headers = {"User-Agent": "Mozilla/5.0"}
r = requests.get(URL, headers=headers)
soup = BeautifulSoup(r.text, "html.parser")

# Lọc link ảnh
"""for img in soup.find_all("img"):
    src = img.get("src") or img.get("data-src") or img.get("data-original")
    if src and any(p in src for p in patterns):
        img_links.append(src)
```
patterns = ["h2.nettruyen", "sf-static.tiktokcdn.com"]
img_links = []
"""
patterns = ["h2.nettruyen", "sf-static.tiktokcdn.com"]
img_links = []

for img in soup.find_all("img"):
    src = img.get("src") or img.get("data-src") or img.get("data-original")
    if src and any(p in src for p in patterns):
        img_links.append(src)

print(f"🎯 Tìm thấy {len(img_links)} ảnh")

# Tải ảnh
for i, link in enumerate(img_links, 1):
    ext = os.path.splitext(urlparse(link).path)[1] or ".jpg"
    filename = f"image-{i:03}{ext}"
    filepath = os.path.join(SAVE_DIR, filename)

    try:
        img_data = requests.get(link, headers=headers).content
        with open(filepath, "wb") as f:
            f.write(img_data)
        print(f"✅ {filename} tải xong")
    except Exception as e:
        print(f"❌ Lỗi tải {link}: {e}")

    time.sleep(3)  # delay 3s tránh bị block

# Nén thành CBZ (nếu cần)
if ZIP_FILE:
    with ZipFile(ZIP_FILE, "w") as zipf:
        for file in os.listdir(SAVE_DIR):
            zipf.write(os.path.join(SAVE_DIR, file), file)
    print(f"📦 Đã tạo {ZIP_FILE}")


SyntaxError: invalid syntax (ipython-input-1024152985.py, line 22)

In [5]:
!pip install beautifulsoup4 requests

In [8]:
import requests
from bs4 import BeautifulSoup
import os
import re
import time
import random
from urllib.parse import urlparse, urljoin
from zipfile import ZipFile
from pathlib import Path

# ==== CONFIG ====
HEADERS = {"User-Agent": "Mozilla/5.0"}
PATTERNS = ["h2.nettruyen", "sf-static.tiktokcdn.com"]
DELAY = (2.5, 4.0)  # delay ngẫu nhiên giữa mỗi ảnh

# ==== HÀM ====

def get_soup(url):
    r = requests.get(url, headers=HEADERS, timeout=30)
    r.raise_for_status()
    return BeautifulSoup(r.text, "html.parser")

def safe_join(base, url):
    if not url:
        return None
    if url.startswith("//"):
        return "https:" + url
    return urljoin(base, url)

def extract_chapter_number(href):
    m = re.search(r"/chuong-(\d+)", href)
    return int(m.group(1)) if m else None

def list_chapters(title_url):
    soup = get_soup(title_url)
    base = "{uri.scheme}://{uri.netloc}".format(uri=urlparse(title_url))
    chapters = {}
    for a in soup.find_all("a", href=True):
        href = safe_join(base, a["href"])
        num = extract_chapter_number(href)
        if num is not None:
            chapters[num] = href
    return sorted(chapters.items())

def find_images(chapter_url):
    soup = get_soup(chapter_url)
    img_links = []
    for img in soup.find_all("img"):
        src = img.get("src") or img.get("data-src") or img.get("data-original")
        if src and any(p in src for p in PATTERNS):
            img_links.append(src)
    return img_links

def download_chapter(chap_num, chap_url, series_dir):
    print(f"\n=== Chapter {chap_num} ===")
    imgs = find_images(chap_url)
    print(f"🖼️ {len(imgs)} ảnh tìm thấy")

    chap_dir = series_dir / f"chapter-{chap_num:03d}"
    chap_dir.mkdir(parents=True, exist_ok=True)

    for i, link in enumerate(imgs, 1):
        ext = os.path.splitext(urlparse(link).path)[1] or ".jpg"
        filename = f"image-{i:03}{ext}"
        filepath = chap_dir / filename
        try:
            img_data = requests.get(link, headers=HEADERS).content
            with open(filepath, "wb") as f:
                f.write(img_data)
            print(f"✅ {filename}")
        except Exception as e:
            print(f"❌ Lỗi {link}: {e}")
        time.sleep(random.uniform(*DELAY))

    # zip -> cbz
    cbz_path = series_dir / f"chapter-{chap_num:03d}.cbz"
    with ZipFile(cbz_path, "w") as zipf:
        for file in sorted(chap_dir.iterdir()):
            zipf.write(file, arcname=file.name)
    print(f"📦 Đã tạo {cbz_path.name}")

def parse_selection(sel, available):
    sel = sel.strip().lower()
    if sel == "all":
        return sorted(available)
    result = set()
    for part in sel.split(","):
        part = part.strip()
        if "-" in part:
            a, b = part.split("-", 1)
            try:
                a, b = int(a), int(b)
                for x in range(min(a, b), max(a, b) + 1):
                    if x in available:
                        result.add(x)
            except:
                pass
        else:
            try:
                x = int(part)
                if x in available:
                    result.add(x)
            except:
                pass
    return sorted(result)

# ==== MAIN ====
title_url = input("Dán link trang truyện (trang tổng): ").strip()
series_slug = Path(urlparse(title_url).path.rstrip("/")).name or "series"
series_dir = Path(series_slug)
series_dir.mkdir(exist_ok=True)

chapters = list_chapters(title_url)
if not chapters:
    print("❌ Không tìm thấy chapter nào.")
    exit()

available_nums = [n for n, _ in chapters]
print(f"Tổng số chapter: {len(available_nums)}")
print(f"Danh sách: {available_nums}")

sel = input("Chọn chapter (all | 1-5 | 1,3,7): ").strip() or "all"
chosen = parse_selection(sel, available_nums)
if not chosen:
    print("❌ Không chọn được chapter hợp lệ.")
    exit()

chap_map = dict(chapters)
for num in chosen:
    download_chapter(num, chap_map[num], series_dir)

print("\n🎉 Hoàn tất!")
print(f"- Ảnh theo chapter: {series_dir}/chapter-XXX")
print(f"- CBZ theo chapter: {series_dir}/chapter-XXX.cbz")


Dán link trang truyện (trang tổng): https://truyentranh3qq.com/vung-hoang-mac
Tổng số chapter: 14
Danh sách: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]
Chọn chapter (all | 1-5 | 1,3,7): 5-14

=== Chapter 5 ===
🖼️ 14 ảnh tìm thấy
✅ image-001.jpg
✅ image-002.jpg
✅ image-003.jpg
✅ image-004.jpg
✅ image-005.jpg
✅ image-006.jpg
✅ image-007.jpg
✅ image-008.jpg
✅ image-009.jpg
✅ image-010.jpg
✅ image-011.jpg
✅ image-012.jpg
✅ image-013.jpg
✅ image-014.jpg
📦 Đã tạo chapter-005.cbz

=== Chapter 6 ===
🖼️ 14 ảnh tìm thấy
✅ image-001.jpg
✅ image-002.jpg
✅ image-003.jpg
✅ image-004.jpg
✅ image-005.jpg
✅ image-006.jpg
✅ image-007.jpg
✅ image-008.jpg
✅ image-009.jpg
✅ image-010.jpg
✅ image-011.jpg
✅ image-012.jpg
✅ image-013.jpg
✅ image-014.jpg
📦 Đã tạo chapter-006.cbz

=== Chapter 7 ===
🖼️ 14 ảnh tìm thấy
✅ image-001.jpg
✅ image-002.jpg
✅ image-003.jpg
✅ image-004.jpg
✅ image-005.jpg
✅ image-006.jpg
✅ image-007.jpg
✅ image-008.jpg
✅ image-009.jpg
✅ image-010.jpg
✅ image-011.jpg
✅ image-012.jpg
